<center>
<h1>Факультет "Прикладная математика" МАИ</h1>
<h2>Курс "Основы Python для анализа данных"</h2>
<h2>Артамонов Игорь Михайлович</h2>
</center>

<center><h2>Занятие № 14. Машинное обучение на текстовых данных.</h2></center>

## Общение / вопросы по курсу

Платформа для групповой работы Atlassian Confluence факультета "Прикладная математика"

https://mai.moscow/display/PYTML

* <b>Занятие № 14. Машинное обучение на текстовых данных</b>

## virtualenv + Jupyter notebook

```
<Ctrl> + <Alt> + T - новое окно терминала
```

```
$ conda -V

$ conda update conda

$ conda search "^python$"

$ conda create -n yourenvname python=x.x anaconda

$ source activate yourenvname

$ jupyter notebook

$ conda install -n yourenvname [package]
```

<b>К прошлой лекции:</b>
    

There are a few discussions for <b>Epoch Vs Iteration</b>.

Iteration is one time processing for forward and backward for a batch of images (say one batch is defined as 16, then 16 images are processed in one iteration).

Epoch is once all images are processed one time individually of forward and backward to the network, then that is one epoch.

I like to make sure my definition of epoch is correct.

One epoch is counted when (Number of iterations * batch size) / total number of images in training

One epoch is counted when Number of iterations == total number of images in training

Which one is correct epoch? My selection is the first one.

---
One iteration means one batch processed. One epoch means all data processed one times.

So one epoch is counted when (batch_size * number_iteration) >= number_data

### <font color="green">ВОПРОС</a>
Какие задачи машинного обучения для __текстов__ могут быть:
* регрессией?
* классификацией?

##   Обработка текстов</h2></center>

#### О чем говорим
* стандартные возможности (их много!)
* частотный подход к тексту:
    - "мешок слов" (bag of words), OHE
    - CV
    - TF/IDF
* "глубокий" разбор текста (не путать с глубокими / глубинными сетями!)
* рекуррентные нейронные сети

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
from numpy.random import randn
import seaborn as sns
from scipy import stats

import matplotlib as mpl
from matplotlib.patches import Ellipse
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score

from sklearn.metrics import accuracy_score 

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [3]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [4]:
import os
import os.path

### Текст 
(как мы его воспринимаем) - это набор:
* слов
* знаков препинания
* нетекстовой информации (например, вставленных изображений или формул) 

### <font color="green">ВОПРОС</font>
Можно ли отнести таблицу к тексту?

### Что может быть важно:
* какие слова?
* последовательность слов
* частота слов
* свойства слов
* последовательность предложений
* текст в целом
* свойства текста

#### Загрузим датасет с 100,000 обзоров курсов с kaggle:<br>
<a href="https://www.kaggle.com/septa97/100k-courseras-course-reviews-dataset">100K Coursera's Course Reviews Dataset</a>

In [5]:
df = pd.read_csv("data14/100k-courseras-course-reviews-dataset/reviews.csv")

In [6]:
df.columns

Index(['Id', 'Review', 'Label'], dtype='object')

In [7]:
df.Label.value_counts()

5    79173
4    18054
3     5071
1     2469
2     2251
Name: Label, dtype: int64

#### Также будем работать отзывами с imdb 

In [17]:
from keras.datasets import imdb

max_features = 10000

print('Loading data...')
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# print(len(x_train), 'train sequences')
# print(len(x_test), 'test sequences')

# save np.load
np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# restore np.load for future normal usage
np.load = np_load_old

Loading data...


In [18]:
np.array(x_train[0])

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223, 5244,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117, 5952,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

### <font color="green">ВОПРОС</font>
Это точно текст?
Если да, то что это всё значит?

## Общие положения

__Порядок__ обработки текстов зависит от __цели__ обработки!

__Возможные операции:__
* удаляем всю (или часть) содержания, не являющееся текстом (картинки, знаки препинания и т.д.)
* переводим в один (обычно - нижний) регистр
* извлекаем информацию с помощью regexp
* нормализуем слова (существительные - единственно число, мужской род, глаголы - в неопределенную форму и т.д)
* делаем встраивание:
    - частотное
    - векторное
* извлекаем информацию из полученного представления текста
* используем полученнц информацию 

<a href="https://www.nltk.org/book/">NLTK Book</href> - https://www.nltk.org/book/

### Стандартные средства работы с текстом в Python
* текст - это не только тексты!
* string - стандартные средства работы со строками
* re - регулярные выражения
* unicodedata - работа с символами UNICODE (уже менее актуально, но ...)
* специфические парсеры:
    - JSON
    - URL (urllib)
    - HTML

## "Встраивание" слов (word embedding)

0                                      good and interesting
1         This class is very helpful to me. Currently, I...
2         like!Prof and TAs are helpful and the discussi...
3         Easy to follow and includes a lot basic and im...
4         Really nice teacher!I could got the point eazl...
                                ...                        
107013    Trendy topic with talks from expertises in the...
107014    Wonderful! Simple and clear language, good ins...
107015     an interesting and fun course. thanks. dr quincy
107016    very broad perspective, up to date information...
107017    An informative course on the social and financ...
Name: Review, Length: 107018, dtype: object

In [25]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

## Преобразуйте Coursera dataset с использованием OneHotEncoder
encoder = OneHotEncoder()
rev = encoder.fit_transform(df.Review)
rev
rev.toarray()

ValueError: Expected 2D array, got 1D array instead:
array=['good and interesting'
 "This class is very helpful to me. Currently, I'm still learning this class which makes up a lot of basic music knowledge."
 'like!Prof and TAs are helpful and the discussion among students are quite active. Very rewarding learning experience!'
 ... 'an interesting and fun course. thanks. dr quincy'
 'very broad perspective, up to date information, useful links and videos and good lecturers in general. Thank you for the insights and knowledge.'
 'An informative course on the social and financial implications due to Zika as well as the factors leading to an epidemic of Zika virus,,'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

#### Слово - в число (обычно - целое)
* уменьшаем объем данных


In [ ]:
from sklearn.preprocessing import LabelEncoder

## Преобразуйте Coursera dataset с использованием LabelEncoder
# Ваш код


#### Слово - в вектор с одной единицей в определенной позицией

Позиционное представление (one-hot encoding (OHE)):
    * одно слово - один столбец
    * если слово есть - то в столбце 1, иначе - 0

In [ ]:
from sklearn.preprocessing import OneHotEncoder

## Преобразуйте Coursera dataset с использованием OneHotEncoder
# Ваш код


### <font color="green">ВОПРОС</font>
* Какой тип выходных данных?
* Почему?
* Что Вы о нём знаете / можете сказать?

### <font color="blue">ПРОБЛЕМА:</blue>
* OHE не обладает смантической близостью<br><br>
Но! __Предложения__ в рамках OHE _могут_ обладать!

<img src="images14/ohe-words-sentense.png"><br><br>
https://medium.freecodecamp.org/text-classification-and-prediction-using-bag-of-words-8aeb1396cded

<img src="images14/bag-of-words-matching.jpg">

## Метрики классификации

Задача классификации состоит в том, чтобы относить ранее неизвестные сущности к тому или иному классу. Примером такой задачи может быть постановка диагноза по медицинским анализам. В этом случае есть два класса результатов: положительный (positive) и отрицательный (negative). Тогда на выходе классификатора может наблюдаться четыре различных ситуации:

* Если результат классификации положительный, и истинное значение тоже положительное, то речь идет об истинно-положительном значении (true-positive, TP)
* Если результат классификации положительный, но истинное значение отрицательное, то речь идет о ложно-положительном значении (false-positive, FP)
* Если результат классификации отрицательный, и истинное значение тоже отрицательное, то речь идет об истинно-отрицательном значении (true-negative, TN)
* Если результат классификации отрицательный, но истинное значение положительно, то речь идет о ложно-отрицательном значении (false-negative, FN)


https://ru.wikipedia.org/wiki/ROC-кривая

$$
Precision = \frac{TP}{TP+FP}
$$<br><br>
$$
Recall = \frac{TP}{TP+FN}
$$<br><br>
F-мера представляет собой гармоническое среднее между точностью и полнотой. Она стремится к нулю, если точность или полнота стремится к нулю.<br><br>
$$
F = 2 \frac{Precision \times Recall}{Precision + Recall}
$$<br><br>
Можно изменить вес точнсти и полноты, придав приоритет одной из этих метрик:<br><br>
$$
F = (\beta^2 + 1) \frac{Precision \times Recall}{Precision + Recall}
$$


### ROC /AUC
<img src="images14/plot_roc_0021.png">

## Частотная модель (вектор встречаемости)

<img src="images14/count_vec.jpeg" width="400">

* с каждым словом текста связывается частота его наличия в тексте
* встречающиеся слова и их количество являются признаком, характеризующим документ

### <font color="blue">ЗАДАНИЕ</font>

* Используя CountVectorizer и логистическую регрессию предскажите классы 
* Для разбения выборки на тренировочную и тестовую воспользуйтесь StratifiedKFold
* В качестве метрики используйте ROC/AUC

### <font color="red">ВАЖНО!</font>
Не забывайте при многоклассовой классификации преобразовывать метки выходного набора с использованием OHE (например, pandas.get_dummies)

In [26]:
# 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score

# Ваш код 
# Настройте параметры векторизатора, если считаете это нужным
cv_vect = CountVectorizer()


### TF-IDF

__Прямая частотность (Term Frequency)__<br><br>

$$
tf_{i,j} = \frac{n_{i,j}}{\sum_k{n_{i,j}}}
$$
__Обратная частотность документов (IDF Inverse Document Frequency)__ <br><br>
$$
idf(w) = log \frac{N}{df_t}
$$
__TF-IDF__<br><br>
$$
w_{i,j} = tf_{i,j} \cdot log (\frac{N}{df_t})
$$

Редкие слова имеют приоритет при классификации

### <font color="blue">ЗАДАНИЕ</font>

* Используя TfidfVectorizer и логистическую регрессию предскажите классы 
* Для разбения выборки на тренировочную и тестовую воспользуйтесь StratifiedKFold
* В качестве метрики используйте ROC/AUC

In [1118]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Создайте TF-IDF
# Настройте векторизатор так, чтобы он использовал одно или 2 слова и количество слов и их сочетаний было 100,000

tf_vect = TfidfVectorizer()
r=[" ".join(review) for review in df['Review'].values]
print(r[0:100])
x = tf_vect.fit_transform(r)  
a=x.toarray()  
tf_vect.get_feature_names()



['g o o d   a n d   i n t e r e s t i n g', "T h i s   c l a s s   i s   v e r y   h e l p f u l   t o   m e .   C u r r e n t l y ,   I ' m   s t i l l   l e a r n i n g   t h i s   c l a s s   w h i c h   m a k e s   u p   a   l o t   o f   b a s i c   m u s i c   k n o w l e d g e .", 'l i k e ! P r o f   a n d   T A s   a r e   h e l p f u l   a n d   t h e   d i s c u s s i o n   a m o n g   s t u d e n t s   a r e   q u i t e   a c t i v e .   V e r y   r e w a r d i n g   l e a r n i n g   e x p e r i e n c e !', 'E a s y   t o   f o l l o w   a n d   i n c l u d e s   a   l o t   b a s i c   a n d   i m p o r t a n t   t e c h n i q u e s   t o   u s e   s k e t c h u p .', 'R e a l l y   n i c e   t e a c h e r ! I   c o u l d   g o t   t h e   p o i n t   e a z l i y   b u t   t h e   v', 'G r e a t   c o u r s e   -   I   r e c o m m e n d   i t   f o r   a l l ,   e s p e c i a l l y   I T   a n d   B u s i n e s s   M a n a g e r s !', 'O n e   o f   t h e   m o s t   u s 

ValueError: empty vocabulary; perhaps the documents only contain stop words

### Векторое представление в виде N-мернго вещественного вектора

<img src="images14/skip-gram.png"><br>
https://towardsdatascience.com/word-embedding-with-word2vec-and-fasttext-a209c1d3e12c

In [1125]:
%ls images14

 ’®¬ ў гбва®©бвўҐ C ­Ґ Ё¬ҐҐв ¬ҐвЄЁ.
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 2269-C3D5

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\Ђ«ҐбЄ ­¤а\Downloads\images14

06.12.2019  14:44    <DIR>          .
06.12.2019  14:44    <DIR>          ..
06.12.2019  14:44            69я920 Backpropagation-Through-Time-BPTT-in-standard-BRNNs.png
06.12.2019  14:44            45я954 bag-of-words-matching.jpg
06.12.2019  14:44            11я324 count_vec.jpeg
06.12.2019  14:44           102я329 greff_lstm_diagram.png
06.12.2019  14:44             6я184 ohe-words-sentense.png
06.12.2019  14:44            80я336 plot_roc_0021.png
06.12.2019  14:44            44я320 rnn.jpg
06.12.2019  14:44           539я494 rnn-description.png
06.12.2019  14:44           180я021 rnn-translation.png
06.12.2019  14:44            56я215 skip-gram.png
              10 д ©«®ў      1я136я097 Ў ©в
               2 Ї Ї®Є  25я873я186я816 Ў ©в бў®Ў®¤­®


In [1126]:
# Возможно, потребуется:

# !pip install --upgrade gensim

In [1132]:
from gensim.models import Word2Vec

# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
    ['this', 'is', 'the', 'second', 'sentence'],
    ['yet', 'another', 'sentence'],
    ['one', 'more', 'sentence'],
    ['and', 'the', 'final', 'sentence']]

# train model
model = Word2Vec(sentences, min_count=1)

# summarize the loaded model
print(model)

# summarize vocabulary
words = list(model.wv.vocab)
print(words)

# access vector for one word
print(model['sentence'])

# save model
model.save('model.bin')

# load model
new_model = Word2Vec.load('model.bin')
print(new_model)


ModuleNotFoundError: No module named 'gensim'

### <font color="blue">ЗАДАНИЕ</font>

Используйте следующий способ для создание вектора сообщения:
* для каждого слова умножте его вектор на значение TF-IDF для этого слова
* усредните эти вектора и примите этот вектор, как вектор сообщения
* вычислите средний вектор по классу
* определите класс сообщения из тестового набора по тому, к какому из векторов классов он ближе всего (используйте косинусное расстояние)

Сделайте это на одном разбиении, полученнос с помощью test_train_split

Пример реализации:
http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

In [ ]:
from sklearn.metrics import pairwise
# исползуйте cosine_similarity 
# Ваш код



## Рекуррентные нейронные сети

* В традиционной NN предполагается, что все входы и выходы независимы друг от друга
* Есть задачи, в которых есть зависимость между предыдущими и последующими данными
* Основная идея - получить возможность работать с последовательной информацией.

<a href="https://www.youtube.com/watch?v=56TYLaQN4N8">https://www.youtube.com/watch?v=56TYLaQN4N8</a> - Highly recommended!

https://www.youtube.com/playlist?list=PLoRl3Ht4JOcdU872GhiYWf6jwrk_SNhz9 - Lecture from the course Neural Networks for Machine Learning, as taught by Geoffrey Hinton (University of Toronto) on Coursera in 2012. 

Link to the course: https://class.coursera.org/neuralnets

<img src="images14/rnn.jpg">
A recurrent neural network and the unfolding in time of the computation involved in its forward computation. Source: Nature

Обучение RNN производится с помощью алгоритма _обратного распространения через время_ (Backpropagation Through Time - ___BPTT___))

<img src="images14/Backpropagation-Through-Time-BPTT-in-standard-BRNNs.png">

<img src="images14/greff_lstm_diagram.png">

Машинный перевод __текстов__ делается с помощью RNN

<img src="images14/rnn-translation.png">
RNN for Machine Translation. Image Source: http://cs224d.stanford.edu/lectures/CS224d-Lecture8.pdf

<img src="images14/rnn-description.png">
Deep Visual-Semantic Alignments for Generating Image Descriptions. Source: http://cs.stanford.edu/people/karpathy/deepimagesent/

In [1272]:
%ls images14

 ’®¬ ў гбва®©бвўҐ C ­Ґ Ё¬ҐҐв ¬ҐвЄЁ.
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : 2269-C3D5

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ C:\Users\Ђ«ҐбЄ ­¤а\Downloads\images14

06.12.2019  14:44    <DIR>          .
06.12.2019  14:44    <DIR>          ..
06.12.2019  14:44            69я920 Backpropagation-Through-Time-BPTT-in-standard-BRNNs.png
06.12.2019  14:44            45я954 bag-of-words-matching.jpg
06.12.2019  14:44            11я324 count_vec.jpeg
06.12.2019  14:44           102я329 greff_lstm_diagram.png
06.12.2019  14:44             6я184 ohe-words-sentense.png
06.12.2019  14:44            80я336 plot_roc_0021.png
06.12.2019  14:44            44я320 rnn.jpg
06.12.2019  14:44           539я494 rnn-description.png
06.12.2019  14:44           180я021 rnn-translation.png
06.12.2019  14:44            56я215 skip-gram.png
              10 д ©«®ў      1я136я097 Ў ©в
               2 Ї Ї®Є  25я744я199я680 Ў ©в бў®Ў®¤­®


In [ ]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')

# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# save np.load
np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# restore np.load for future normal usage
np.load = np_load_old


print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

<br>
<b>Это хороший результат ???</a>


## Нейронные сети

В качестве домашнего задания:
* разберитесь с ядром с kaggle 
    - https://www.kaggle.com/reiinakano/basic-nlp-bag-of-words-tf-idf-word2vec-lstm
* попробуйте реализовать то же самое с Medium dataset

На следующее занятие:
* внимательно повторите и почитайте про RNN
* они потребуются нам для временных рядов (!)

In [ ]:
# Ваш код


https://github.com/Prakashvanapalli/TensorFlow/blob/master/Blogposts/Backpropogation_with_Images.ipynb

## Экзаменационные вопросы:

* Частотная модель текста
* TF/IDF
* Векторное представление слов и текстов
* Рекуррентные нейронны сети